## Meteo forcing example

This is an example of reading locally stored meteo files from HNMS in Rotated_II format. The data are resamplbed on WGS84 grid with pyresample. That way they can be used in the Hydrodynamic solvers. More info in the correspoding Notebook.

In [ ]:
#to use the full width of the browser window
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import pyPoseidon
import pyPoseidon.meteo as pmeteo
import datetime
import geoviews.feature as gf
import holoviews as hv
import geoviews as gv
import glob

In [ ]:
hv.notebook_extension('bokeh')

In [ ]:
pyPoseidon.__version__

### setup

In [ ]:
#define in a dictionary the properties of the model..
dic={'minlon':-5.5, # lat/lon window
     'maxlon':42.,
     'minlat':30,
     'maxlat':47.5,
     'start_date':'2018-1-1',
     'end_date':'2018-1-1 12'
    }

In [ ]:
#creating a sequence of folder from which we read the meteo. Adapt accordingly 
PATH='/Users/brey/Downloads/HNMS/01/'
meteo = glob.glob(PATH+'/E_JRC00*')
meteo.sort()
meteo

In [ ]:
dic.update({'mpaths':meteo[:12]}) # select relevant range

### retrieve UVP

In [ ]:
#get uvp
#meteo = ecmwf_oper(**dic)
#meteo = generic(filename='/Users/brey/Downloads/wrf_uvp.nc')
meteo = pmeteo.hnms_oper(**dic)

In [ ]:
meteo.uvp

## Visualise

### holoviews

In [ ]:
%%opts Image [width=650 height=450] (cmap='viridis')
meteo.hview.to(hv.Image, ['longitude','latitude'], 'msl').hist()

### geoviews

In [ ]:
g50 = gv.feature.coastline(plot=dict(scale='50m'), style=dict(linewidth=1.5))

In [ ]:
%%opts Image [colorbar=True width=650 height=450 toolbar="above"  tools=['hover']] (cmap='viridis')
#meteo.gview.to(gv.Image, ['longitude','latitude'], 'msl', dynamic=True) * g50 * gf.borders()
meteo.gview.to(gv.Image, ['longitude','latitude'], 'msl') * g50 * gf.borders()

## Output

In [ ]:
#generic netCDF
meteo.uvp.to_netcdf('./test/test.nc') # netcdf

In [ ]:
#for D3D solver
meteo.output(solver='d3d',rpath='./test/') # u,v,p for d3d

In [ ]:
#save to SCHISM format
meteo.output(solver='schism',rpath='./test/')